In [ ]:
!pip install tensorflow==2.2.0
!pip install sklearn --upgrade

In [1]:
import os
import sqlite3
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D, Flatten, Dense
import pandas as pd
import numpy as np
from flask import Flask, jsonify, render_template, request
from tensorflow.keras.models import load_model
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder

df_test = pd.read_csv("mushroomstest.csv")

df_test.head()




,class,cap-shape,cap-surface,cap-color,odor,spore-print-color,population
0,1,2,3,0,7,0,3
1,0,2,3,9,0,1,2
2,0,0,3,8,1,1,2
3,1,2,2,8,7,0,3
4,0,2,3,3,6,1,0


In [2]:
# load model
model = load_model('model/mushrooms_deeplearn2.h5')
# summarize model.
model.summary()

from sklearn.preprocessing import MinMaxScaler

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 100)               700       
_________________________________________________________________
dense_4 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_5 (Dense)              (None, 2)                 202       
Total params: 11,002
Trainable params: 11,002
Non-trainable params: 0
_________________________________________________________________


In [3]:
#if request.method == 'POST':
    #mushroom = request.form['mushroom']
    


df = pd.DataFrame (df_test, columns = ['cap-shape', 'cap-surface', 'cap-color', 'odor', 'spore-print-color', 'population'])
#for index,row in df.itterrows(): 
    #X_scaler = MinMaxScaler().fit(row)

X_scaler = MinMaxScaler().fit(df)
test_scaled = X_scaler.transform(df)
encoded_predictions= model.predict_classes(test_scaled)
if encoded_predictions[0] == 1:
    final_prediction = "Likely Poisonous"
else: 
    final_prediction = "Likely Safe"
    
#save as sql lite or json 


Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).


In [4]:
print(encoded_predictions)
print(type(encoded_predictions))

[1 0 0 1 0 0 0 0 1 0 0 0 0 1 0 0 0 1 1 1 0 1]
<class 'numpy.ndarray'>


In [19]:
predictList= []
for i in encoded_predictions:
   
    if i == 1:
        final_prediction = "Likely Poisonous"
    else: 
        final_prediction = "Likely Safe"
    print (final_prediction)
    predictList.append(final_prediction)
    #print(encoded_predictions)
    
#print(final_prediction)

Likely Poisonous
Likely Safe
Likely Safe
Likely Poisonous
Likely Safe
Likely Safe
Likely Safe
Likely Safe
Likely Poisonous
Likely Safe
Likely Safe
Likely Safe
Likely Safe
Likely Poisonous
Likely Safe
Likely Safe
Likely Safe
Likely Poisonous
Likely Poisonous
Likely Poisonous
Likely Safe
Likely Poisonous


In [20]:
print(predictList)

['Likely Poisonous', 'Likely Safe', 'Likely Safe', 'Likely Poisonous', 'Likely Safe', 'Likely Safe', 'Likely Safe', 'Likely Safe', 'Likely Poisonous', 'Likely Safe', 'Likely Safe', 'Likely Safe', 'Likely Safe', 'Likely Poisonous', 'Likely Safe', 'Likely Safe', 'Likely Safe', 'Likely Poisonous', 'Likely Poisonous', 'Likely Poisonous', 'Likely Safe', 'Likely Poisonous']


In [21]:

predictDF=pd.DataFrame(predictList, index =[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21], 
                                              columns =['Predicition'])
predictDF

,Predicition
0,Likely Poisonous
1,Likely Safe
2,Likely Safe
3,Likely Poisonous
4,Likely Safe
5,Likely Safe
6,Likely Safe
7,Likely Safe
8,Likely Poisonous
9,Likely Safe


In [22]:
# Export file as a CSV, without the Pandas index, but with the header
predictDF.to_csv("sample_prediction.csv", index=True, header=True)

In [23]:
import csv_to_sqlite 
# all the usual options are supported
options = csv_to_sqlite.CsvOptions(typing_style="full", encoding="utf-8") 
input_files = ["sample_prediction.csv"] # pass in a list of CSV files
csv_to_sqlite.write_csv(input_files, "mushroom_prediction.sqlite", options)

Written 22 rows into 1 tables in 0.020 seconds


22

In [24]:
conn = sqlite3.connect("mushroom_prediction.sqlite")

In [25]:
cur = conn.cursor()
cur.execute("SELECT * FROM sample_prediction")
cur.fetchall()

[(0, 'Likely Poisonous'),
 (1, 'Likely Safe'),
 (2, 'Likely Safe'),
 (3, 'Likely Poisonous'),
 (4, 'Likely Safe'),
 (5, 'Likely Safe'),
 (6, 'Likely Safe'),
 (7, 'Likely Safe'),
 (8, 'Likely Poisonous'),
 (9, 'Likely Safe'),
 (10, 'Likely Safe'),
 (11, 'Likely Safe'),
 (12, 'Likely Safe'),
 (13, 'Likely Poisonous'),
 (14, 'Likely Safe'),
 (15, 'Likely Safe'),
 (16, 'Likely Safe'),
 (17, 'Likely Poisonous'),
 (18, 'Likely Poisonous'),
 (19, 'Likely Poisonous'),
 (20, 'Likely Safe'),
 (21, 'Likely Poisonous')]